# LangChain v1: 1. ReAct Agent with Multiple Tools (QA, Summarization & Web Search)

# 🧠 1️⃣ ReAct Agent (Reason + Act) ⭐ MOST COMMON

🔍 What it does
*    Thinks step-by-step
*    Chooses the right tool
*    Executes the tool
*    Gives final answer

Flow
* Think → Choose Tool → Use Tool → Answer

✅ Use when
- You have multiple tools
- Path is not known in advance
- AI must decide dynamically

🧩 Small Example
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(
    model=llm,
    tools=[simple_qa, summarizer, web_search]
)

Example question
- "Latest update about GPT-4o"

Agent decision
- → Use web_search tool

🧠 Mental model
- Smart assistant that thinks before acting

🏆 Summary
- 🔥 Default & recommended agent
- 🧠 Thinks + uses tools
- 🛠 Most real projects use this

Final Comparison
| Agent Type       | Best For      | Thinks? | Tools? |
| ---------------- | ------------- | ------- | ------ |
| * ReAct            | General AI    | ✅       | ✅      |
| Tool-Calling     | APIs          | ❌       | ✅      |
| Planner-Executor | Complex tasks | ✅       | ✅      |
| Router           | Multi-agent   | 🟡      | 🟡     |
| Human-in-Loop    | Safety        | 🟡      | 🟡     |


In [4]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

# =====================================================
# 🔐 Load API keys
# =====================================================
load_dotenv(".env")

# =====================================================
# 🔸 Initialize LLM
# =====================================================
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# =====================================================
# ✅ GLOBAL INSTRUCTION (used everywhere)
# =====================================================
SHORT_INSTRUCTION = "Give a SHORT and CLEAR answer. Avoid long explanations."

# =====================================================
# ✅ Tool 1: Simple QA Tool
# =====================================================
qa_prompt = PromptTemplate.from_template(
    "Question: {question}\nInstruction: {instruction}"
)
qa_chain = qa_prompt | llm

@tool
def simple_qa(question: str) -> str:
    """Answer factual questions briefly."""
    response = qa_chain.invoke({
        "question": question,
        "instruction": SHORT_INSTRUCTION
    })
    return response.content.strip()

# =====================================================
# ✅ Tool 2: Summarizer Tool
# =====================================================
summary_prompt = PromptTemplate.from_template(
    "Text:\n{text}\n\nInstruction: {instruction}"
)
summary_chain = summary_prompt | llm

@tool
def summarizer(text: str) -> str:
    """Summarize text briefly."""
    response = summary_chain.invoke({
        "text": text,
        "instruction": SHORT_INSTRUCTION
    })
    return response.content.strip()

# =====================================================
# ✅ Tool 3: Web Search Tool (Tavily)
# =====================================================
tavily_search = TavilySearchResults(max_results=3)

@tool
def web_search(query: str) -> str:
    """Search the internet and return a brief answer."""
    results = tavily_search.invoke({"query": query})

    # Keep search output short
    summaries = []
    for r in results:
        summaries.append(f"- {r.get('title', '')}")

    return "\n".join(summaries[:3])

# =====================================================
# 🧠 Create Agent
# =====================================================
tools = [simple_qa, summarizer, web_search]

agent = create_react_agent(
    model=llm,
    tools=tools
)

# =====================================================
# 🚀 Run user queries
# =====================================================
queries = [
    "What is LangGraph in LangChain?",
    "Summarize this: LangChain helps build LLM apps with tools and agents.",
    "Latest update about Langchain"
]

for query in queries:
    print("\n🧑‍💻 User Query:", query)
    response = agent.invoke({"messages": [("user", query)]})
    print("\n🤖 Agent Response:", response["messages"][-1].content)



C:\Users\15016\AppData\Local\Temp\ipykernel_35184\1423584563.py:84: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(



🧑‍💻 User Query: What is LangGraph in LangChain?

🤖 Agent Response: LangGraph in LangChain is a framework that enables users to create and manage complex workflows by connecting various language model components and tasks in a graph-like structure.

🧑‍💻 User Query: Summarize this: LangChain helps build LLM apps with tools and agents.

🤖 Agent Response: LangChain is a framework for building applications using large language models (LLMs) with tools and agents.

🧑‍💻 User Query: Latest update about Langchain

🤖 Agent Response: LangChain has recently achieved a valuation of $1.25 billion, officially making it a unicorn. This reflects the increasing interest and investment in AI orchestration technologies.


In [3]:
for query in queries:
    print("\n🧑‍💻 User Query:", query)

    response = agent.invoke({"messages": [("user", query)]})

    # Final answer
    print("\n🤖 Final Answer:", response["messages"][-1].content)

    # Tool usage
    used_tools = [
        msg.name for msg in response["messages"]
        if msg.type == "tool"
    ]
    print("🛠 Tools used:", used_tools)



🧑‍💻 User Query: What is LangGraph in LangChain?

🤖 Final Answer: LangGraph in LangChain is a framework that enables users to create and manage complex workflows by connecting various language model components and data sources in a graph-like structure.
🛠 Tools used: ['simple_qa']

🧑‍💻 User Query: Summarize this: LangChain helps build LLM apps with tools and agents.

🤖 Final Answer: LangChain is a framework for building applications using large language models (LLMs) with tools and agents.
🛠 Tools used: ['summarizer']

🧑‍💻 User Query: Latest update about GPT-4o

🤖 Final Answer: The latest update about GPT-4o indicates that its final day is set for February 16th, 2026. Additionally, there has been a deprecation of the chat-gpt-4o-latest version, and discussions are ongoing about the potential retirement of GPT-4 in 2026, along with its implications.
🛠 Tools used: ['web_search']
